In [10]:
import psycopg2
import subprocess
from psycopg2.extensions import ISOLATION_LEVEL_AUTOCOMMIT

In [2]:
hostname = 'localhost'
username = 'qfixuser'
database = 'qfix'
password = 'QFIXDBpassword'

In [3]:
def alterConfigFile(numRows,numQueries,configFile='/home/mj1e16/abduction/pwe/testData/qfixNew/qfix_sync/miniconfig.txt'):
    N_D = 'N_D: '
    N_q = 'N_q: '
    endLine = '\n'
    with open(configFile,'r') as f:
        data =  f.read()
    loc1 = data.find(N_D)
    loc2 = data[loc1:].find(endLine) + loc1

    dataNew = data[:loc1]+N_D+str(numRows)+data[loc2:]

    loc3 = dataNew.find(N_q)
    loc4 = dataNew[loc3:].find(endLine) + loc3

    dataFinal = dataNew[:loc3]+N_q+str(numQueries)+dataNew[loc4:]
    with open(configFile,'w') as f:
        f.write(dataFinal)

In [8]:
def alterRunCommands(commandFile,commandOut,database):
    with open(commandFile,'r') as f:
        data = f.read()
    dataNew = data.replace('<dbName>',database)
    with open(commandOut,'w') as f:
        f.write(dataNew)

In [11]:
def createDataBase(dbName,hostname='localhost',username='qfixuser',database='qfix',password='QFIXDBpassword'):
    myConnection = psycopg2.connect( host=hostname, user=username, password=password, dbname=database )
    myConnection.set_isolation_level(ISOLATION_LEVEL_AUTOCOMMIT)
    curr = myConnection.cursor()
    curr.execute('create database "{}";'.format(dbName))
    

In [13]:
def postgre2CSVQuerys(tableName,fileName,hostname='localhost',username='qfixuser',database='qfix',password='QFIXDBpassword'):
    myConnection = psycopg2.connect( host=hostname, user=username, password=password, dbname=database )
    curr = myConnection.cursor()
    curr.execute('SELECT * FROM {};'.format(tableName))
    rows = curr.fetchall()
    queryList = []
    for row in rows:
        queryList.append(row[-1])
    queryString = '\n'.join(queryList)
    with open(fileName,'w') as f:
        f.write(queryString)

In [14]:
def postgre2CSVData(tableName,fileName,hostname='localhost',username='qfixuser',database='qfix',password='QFIXDBpassword'):
    myConnection = psycopg2.connect( host=hostname, user=username, password=password, dbname=database )
    curr = myConnection.cursor()
    curr.execute('SELECT * FROM {};'.format(tableName))
    rows = curr.fetchall()
    stringRows = [str(x) for x in rows]
    queryString = '\n'.join(stringRows)
    with open(fileName,'w') as f:
        f.write(queryString)

In [15]:
def postgreDropTable(tableName,hostname='localhost',username='qfixuser',database='qfix',password='QFIXDBpassword'):
    myConnection = psycopg2.connect( host=hostname, user=username, password=password, dbname=database )
    curr = myConnection.cursor()
    #for tab in tableNames:
    curr.execute('DROP TABLE IF EXISTS {};'.format(tableName))
    

In [20]:
N_rows = [50,100,500]
N_queries = 50

In [21]:
queryName = []
dataName = []
for n in N_rows:
    dbName = 'qfixedRows{}Queries{}'.format(n,N_queries)
    alterConfigFile(n,N_queries)
    alterRunCommands('/home/mj1e16/abduction/pwe/testData/qfixNew/qfix_sync/runCommandsBase.sh','/home/mj1e16/abduction/pwe/testData/qfixNew/qfix_sync/runCommands.sh',dbName)
    createDataBase(dbName)
    subprocess.call(['/home/mj1e16/abduction/pwe/testData/qfixNew/qfix_sync/runCommands.sh'])
    postgre2CSVData('synth_1_clean_0','./qfixData_Nrows{}_Nqueries{}.csv'.format(n,N_queries),database=dbName)
    postgre2CSVQuerys('qlogs','qfixQueries_Nrows{}_Nqueries{}.csv'.format(n,N_queries),database=dbName)
    queryName.append('/home/mj1e16/abduction/pwe/jupyterNotebooks/notInPWEenv/qfixData_Nrows{}_Nqueries{}.csv'.format(n,N_queries))
    dataName.append('/home/mj1e16/abduction/pwe/jupyterNotebooks/notInPWEenv/qfixQueries_Nrows{}_Nqueries{}.csv'.format(n,N_queries))

In [22]:
print(queryName)
print(dataName)

['/home/mj1e16/abduction/pwe/jupyterNotebooks/notInPWEenv/qfixData_Nrows50_Nqueries50.csv', '/home/mj1e16/abduction/pwe/jupyterNotebooks/notInPWEenv/qfixData_Nrows100_Nqueries50.csv', '/home/mj1e16/abduction/pwe/jupyterNotebooks/notInPWEenv/qfixData_Nrows500_Nqueries50.csv']
['/home/mj1e16/abduction/pwe/jupyterNotebooks/notInPWEenv/qfixQueries_Nrows50_Nqueries50.csv', '/home/mj1e16/abduction/pwe/jupyterNotebooks/notInPWEenv/qfixQueries_Nrows100_Nqueries50.csv', '/home/mj1e16/abduction/pwe/jupyterNotebooks/notInPWEenv/qfixQueries_Nrows500_Nqueries50.csv']


In [24]:
postgre2CSVData('synth_1_clean_0','./testData_Nrows{}_Nqueries{}.csv'.format(n,N_queries))

In [26]:
postgre2CSVQuerys('qlogs','testQueries.csv')